In [ ]:
# Install required python packages
!pip install oauth2client
!pip install google-api-python-client
!pip install httplib2

In [ ]:
# Import required packages
from oauth2client.client import OAuth2WebServerFlow
from googleapiclient.discovery import build
import httplib2

In [ ]:
# Use Your Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "Your client ID"
CLIENT_SECRET = "Your client secret"

# Define Oath scopes with read only access
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"

#OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters"

# Redirect URI to open Authorization Code Window in Browser
REDIRECT_URI = 'Redirect URI'

In [ ]:
# Build URL that generates Authorization Code
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()

#Print Authorization URL
print("Go to the following link in your browser: " + authorize_url)

In [ ]:
auth_code = input("Enter your Authorization Code here:")

In [ ]:
#get credentials to build GSC API service
credentials = flow.step2_exchange(auth_code)
print("The credentials are generated successfully")

In [ ]:
# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()

#authorize credentials
creds = credentials.authorize(http)

#building a service to access various features of GSC API
webmasters_service = build('searchconsole', 'v1', http=creds)

    List of sites for my google account

In [ ]:
site_list = webmasters_service.sites().list().execute()

print(site_list)

In [ ]:
# Using a list comprehension to extract site URLs
site_urls = [entry['siteUrl'] for entry in site_list['siteEntry']]

# Print the list of site URLs
print(site_urls)

In [ ]:
# Create a DataFrame from the site URLs
df = pd.DataFrame({'Site URLs': site_urls})

# Define the Excel file name
excel_file = 'site_urls.xlsx'

# Export the DataFrame to an Excel file
df.to_excel(excel_file, index=False)

print(f"Site URLs have been saved to '{excel_file}'.")

    Search analytics data with row limit of 25000

In [ ]:
# the website we want to get the data for
website = "https://www.knorr.com/uk/"

# build a request body
request_body = {
    "startDate"	: '2023-01-01',
    "endDate" : '2023-02-28',
    "dimensions" : ['QUERY', 'PAGE'],
    "rowLimit" : 25000,
    "dataState" : "final"
}

# get the response using request body
response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()
print(response_data)

In [ ]:
len(response_data['rows'])

In [ ]:
for each in response_data['rows']:
  print(each)

Modified script to append to existing data - Knorr UK

In [ ]:
# the website we want to get the data for
website = "https://www.knorr.com/uk/"

# create an empty list to store the rows from response
all_responses = []

# define a startRow
startRow = 0

while (startRow == 0) or (startRow%25000 == 0):
  # build a request body
  request_body = {
      "startDate"	: '2023-01-01',
      "endDate" : '2023-01-02',
      "dimensions" : ['DATE','QUERY', 'PAGE'],
      "rowLimit" : 25000,
      "dataState" : "final",
      'startRow' : startRow
      }

  #get gsc response
  response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

  #update the rows
  startRow = startRow + len(response_data['rows'])
  print("fetched up to " + str(startRow) + " rows of data")

  # for loop tos save all the rows in all_responses
  for row in response_data['rows']:
    all_responses.append(row)

In [ ]:
len(all_responses)

In [ ]:
all_responses

    Build a Dataframe using Pandas

In [ ]:
import pandas as pd

In [ ]:
#empty list to build a dataframe
data_for_df = []

for each in all_responses:

  temp = []

  #date
  temp.append(each['keys'][0])

  #query
  temp.append(each['keys'][1])

  #page
  temp.append(each['keys'][2])

  #clicks
  temp.append(each['clicks'])

  #impressions
  temp.append(each['impressions'])

  #ctr
  temp.append(each['ctr'])

  #position
  temp.append(each['position'])

  data_for_df.append(temp)



In [ ]:
data_for_df

In [ ]:
df = pd.DataFrame(data_for_df, columns=['date', 'query', 'page', 'clicks', 'impressions', 'ctr', 'position'])

In [ ]:
df

    Export to CSV or Excel

In [ ]:
df.to_excel('GSC data 2.xlsx')

In [ ]:
df.to_csv('GSC data 2.csv')

    Site by site

In [ ]:
# the website we want to get the data for
website = "https://www.hellmanns.com/us/es/"

# Define the CSV file path
csv_file_path = "C:/Users/narkamat/GSC_data_4.csv"

# create an empty list to store the rows from response
all_responses = []

# define a startRow
startRow = 0

while (startRow == 0) or (startRow%25000 == 0):
  # build a request body
  request_body = {
      "startDate"	: '2023-08-01',
      "endDate" : '2023-10-31',
      "dimensions" : ['DATE', 'QUERY', 'PAGE'],
      "rowLimit" : 25000,
      "dataState" : "final",
      'startRow' : startRow
      }

  #get gsc response
  response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

  #update the rows
  startRow = startRow + len(response_data['rows'])
  print("fetched up to " + str(startRow) + " rows of data")

  # for loop tos save all the rows in all_responses
  for row in response_data['rows']:
    all_responses.append(row)
    
    
#empty list to build a dataframe
data_for_df = []

for each in all_responses:

  temp = []

  #date
  temp.append(each['keys'][0])

  #query
  temp.append(each['keys'][1])

  #page
  temp.append(each['keys'][2])

  #clicks
  temp.append(each['clicks'])

  #impressions
  temp.append(each['impressions'])

  #ctr
  temp.append(each['ctr'])

  #position
  temp.append(each['position'])

  data_for_df.append(temp)

# Load the existing CSV file into a DataFrame if it exists, or create a new one
try:
    df = pd.read_csv(csv_file_path)
except FileNotFoundError:
    df = pd.DataFrame(columns=['date', 'query', 'page', 'clicks', 'impressions', 'ctr', 'position'])

# Create a new DataFrame for the current website's data
new_df = pd.DataFrame(data_for_df, columns=['date', 'query', 'page', 'clicks', 'impressions', 'ctr', 'position'])

# Append the new data to the existing DataFrame
df = pd.concat([df, new_df], ignore_index=True)

# Save the updated DataFrame to the CSV file
df.to_csv(csv_file_path, index=False)

print(f'Data from {website} appended to {csv_file_path}')